<a href="https://colab.research.google.com/github/Mayaa722/Ai-safety-system/blob/main/Non_Fire_objects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium undetected-chromedriver
!pip install selenium

In [ ]:
!apt-get update
!apt-get install -y wget unzip
!apt-get remove -y chromium-chromedriver

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,207 B]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,032 B in 3s (1,195 B/s)
Reading package lists... Done
W: Skipping a

In [ ]:
!pip install webdriver-manager selenium


In [ ]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

--2025-11-30 12:01:38--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.251.108.190, 142.251.108.91, 142.251.108.93, ...
Connecting to dl.google.com (dl.google.com)|142.251.108.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117885136 (112M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb.1’

google-chrome-stabl 100%[===================>] 112.42M  23.3MB/s    in 5.6s    

2025-11-30 12:01:44 (20.2 MB/s) - ‘google-chrome-stable_current_amd64.deb.1’ saved [117885136/117885136]

(Reading database ... 122141 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (142.0.7444.175-1) over (142.0.7444.175-1) ...
Setting up google-chrome-stable (142.0.7444.175-1) ...
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
Reading pac

In [ ]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

cp: cannot stat '/usr/lib/chromium-browser/chromedriver': No such file or directory


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.binary_location = "/usr/bin/google-chrome-stable"


In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


In [ ]:
driver.get("https://www.google.com")
print(driver.title)
driver.quit()

Google


In [ ]:
import os
import time
import requests
from selenium.webdriver.common.by import By

In [ ]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# ---------------------------
# CONFIGURATION
# ---------------------------
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

SEARCH_SITES = {
    "unsplash": "https://unsplash.com/s/photos/{}",
    "pexels": "https://www.pexels.com/search/{}/",
    "pixabay": "https://pixabay.com/images/search/{}/",
    "vecteezy": "https://www.vecteezy.com/free-photos?q={}",
    "mixkit": "https://mixkit.co/free-stock-photos/{}/",
    "gettyimages": "https://www.gettyimages.com/photos/{}"
}

# Folder to save images
DOWNLOAD_FOLDER = "violence_images"
os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)

# Exclude unrelated content
NEGATIVE_FILTER = [
    "cat", "dog", "tree", "flower", "food", "car", "house",
    "boxing", "kickboxing", "karate", "taekwondo", "martial arts"
]

# Violence/fight queries
violence_queries = [
    "street fight",
    "physical fight",
    "combat training",
    "self defense",
    "sparring fight",
    "violent confrontation",
    "fist fight",
    "wrestling fight",
    "aggression",
    "brawl"
]

# ---------------------------
# SCRAPER FUNCTION
# ---------------------------
def scrape_violence(queries, target_count=150, delay=2.0):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    all_downloaded_urls = set()
    total_downloaded = 0

    try:
        for query in queries:
            safe_query = query.replace(" ", "_").lower()
            downloaded = 0

            for site_name, base_url in SEARCH_SITES.items():
                print(f"\n--- Searching {site_name} for '{query}' ---")
                search_url = base_url.format(query.replace(" ", "-"))

                try:
                    driver.get(search_url)
                except:
                    try:
                        driver.quit()
                    except:
                        pass
                    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
                    driver.get(search_url)

                time.sleep(delay + random.random()*0.5)

                images = driver.find_elements(By.TAG_NAME, "img")
                for img in images:
                    if downloaded >= target_count:
                        break
                    src = img.get_attribute("src")
                    if not src:
                        continue

                    # Skip unrelated content
                    if any(x in src.lower() for x in NEGATIVE_FILTER):
                        continue

                    # GettyImages specific filter
                    if site_name == "gettyimages" and "media.gettyimages.com" not in src:
                        continue

                    # Skip duplicates
                    if src in all_downloaded_urls:
                        continue

                    all_downloaded_urls.add(src)

                    # Save image
                    filename = f"{safe_query}_{total_downloaded+1}.jpg"
                    path = os.path.join(DOWNLOAD_FOLDER, filename)

                    success = False
                    for attempt in range(3):
                        try:
                            r = requests.get(src, timeout=10)
                            if r.status_code == 200:
                                with open(path, "wb") as f:
                                    f.write(r.content)
                                downloaded += 1
                                total_downloaded += 1
                                success = True
                                print(f"Downloaded: {path} ({downloaded}/{target_count})")
                                break
                        except:
                            time.sleep(1)
                    if not success:
                        print("Failed:", src)

                if downloaded >= target_count:
                    break
            print(f"Finished query '{query}' with {downloaded} images.")

    finally:
        driver.quit()

    return all_downloaded_urls

# ---------------------------
# RUN SCRAPER
# ---------------------------
results = scrape_violence(violence_queries, target_count=150, delay=2.0)
print(f"\nTotal unique images downloaded: {len(results)}")

# ---------------------------
# ZIP DATASET
# ---------------------------
shutil.make_archive(DOWNLOAD_FOLDER, 'zip', DOWNLOAD_FOLDER)
print(f"ZIP created: {DOWNLOAD_FOLDER}.zip")

# ---------------------------
# COLAB DOWNLOAD (optional)
# ---------------------------
try:
    from google.colab import files
    files.download(DOWNLOAD_FOLDER + ".zip")
except ImportError:
    print("Not running in Colab. Zip is at:", DOWNLOAD_FOLDER + ".zip")



--- Searching unsplash for 'street fight' ---
Failed: https://unsplash.com/.within.website/x/cmd/anubis/static/img/reject.webp?cacheBuster=v1.23.1

--- Searching pexels for 'street fight' ---

--- Searching pixabay for 'street fight' ---

--- Searching vecteezy for 'street fight' ---

--- Searching mixkit for 'street fight' ---
Failed: data:,
Failed: 
Downloaded: violence_images/street_fight_1.jpg (1/150)
Downloaded: violence_images/street_fight_2.jpg (2/150)
Downloaded: violence_images/street_fight_3.jpg (3/150)
Downloaded: violence_images/street_fight_4.jpg (4/150)

--- Searching gettyimages for 'street fight' ---
Downloaded: violence_images/street_fight_5.jpg (5/150)
Downloaded: violence_images/street_fight_6.jpg (6/150)
Downloaded: violence_images/street_fight_7.jpg (7/150)
Downloaded: violence_images/street_fight_8.jpg (8/150)
Downloaded: violence_images/street_fight_9.jpg (9/150)
Downloaded: violence_images/street_fight_10.jpg (10/150)
Downloaded: violence_images/street_fight_11

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
import shutil

In [ ]:
import shutil
shutil.make_archive("fight_photos", 'zip', "fight_photos")
print("ZIP created: fight_photos.zip")

# Trigger download in Colab
try:
    from google.colab import files
    files.download("fight_photos.zip")
except ImportError:
    print("Not running in Colab. Zip is at:", "fight_photos.zip")

ZIP created: fight_photos.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>